In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv2D, Flatten, Dropout, MaxPooling2D, MaxPooling3D, UpSampling2D, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras import backend as K

In [ ]:
X_train = np.load('../y_smp_train.npy')
y_train = np.load('../pars_smp_train.npy')
X_test = np.load('../y_smp_test.npy')

In [ ]:
X_train.shape, y_train.shape

In [ ]:
#!g2.1
def quantile_loss_9(y_true, y_pred):
    error = y_true - y_pred
    loss = K.mean(K.maximum(0.9 * error, (0.9 - 1) * error))
    return loss

In [ ]:
#!g2.1
time_steps = 200
num_features = 3

In [ ]:
#!g2.1
model = Sequential()

model.add(Conv2D(16, (3,3), activation='relu', input_shape=(200, 3, 1), padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32, (12,6), activation='relu', strides=(6, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', strides=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dense(15))

opt = AdamW(learning_rate=0.0001)
model.compile(loss=quantile_loss_9, optimizer=opt, metrics=[RootMeanSquaredError()])

In [ ]:
#!g2.1
model.summary()

In [ ]:
#!g2.1
model.fit(X_train, y_train, batch_size=128, epochs=100)

In [ ]:
#!g2.1
pred = model.predict(X_test)

In [ ]:
#!g2.1
np.save('test_pred_09', pred)

In [ ]:
#!g2.1
model.save("model_09.h5")